In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

In [30]:
data = pd.read_csv('../data_file/fix_holiday_feature_train.csv')
test_data = pd.read_csv('../data_file/fix_holiday_feature_test.csv')
submission = pd.read_csv('../data_file/sample_submission.csv')

/var/folders/__/r8r58jz17gj32tjwycl0m0hr0000gn/T/ipykernel_80346/4194602010.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data_file/fix_holiday_feature_train.csv')


In [39]:
len(test_data), len(submission), len(data)

(16800, 16800, 201788)

In [41]:
len(data) + len(test_data)

218588

In [32]:
def process_building_data(train_data, test_data, building_num):
    """Revised process for the data of a specific building number."""
    
    # Filter the datasets for the specific building number
    train_building = train_data[train_data['building_number'] == building_num]
    test_building = test_data[test_data['building_number'] == building_num]
    
    # Concatenate the train and test datasets for the building
    concatenated_building = pd.concat([train_building, test_building], axis=0, ignore_index=True)
    
    # Backup the 'power_consumption' column
    power_consumption_backup = concatenated_building['power_consumption'].copy()
    
    # Shift all columns 4 hours ahead
    concatenated_building = concatenated_building.shift(-4)
    
    # Restore the 'power_consumption' column to its original state
    concatenated_building['power_consumption'] = power_consumption_backup
    
    # For the last 4 rows, fill the specified columns with values from 24 hours earlier
    columns_to_shift = ['cos_time', 'sin_time', 'peak', 'no_peak']
    for column in columns_to_shift:
        concatenated_building[column].iloc[-4:] = concatenated_building[column].iloc[-28:-24].values
    
    # For the last 4 rows, fill other columns (except 'temperature') with the values from the last available row
    last_row = concatenated_building.iloc[-5].copy()
    for column in concatenated_building.columns:
        if column not in columns_to_shift and column != 'temperature':
            concatenated_building[column].iloc[-4:] = last_row[column]
    
    # Predict the temperature for the last 4 rows
    features = ['rainfall', 'windspeed', 'humidity', 'sin_time', 'cos_time']
    X = concatenated_building.dropna(subset=['temperature'])[features]
    y = concatenated_building.dropna(subset=['temperature'])['temperature']
    
    # Train the linear regression model
    regressor = LinearRegression()
    regressor.fit(X, y)
    
    # Predict the temperature for the last 4 rows
    predicted_temps = regressor.predict(concatenated_building[features].iloc[-4:])
    
    # Fill the NaN values in the temperature column with the predicted values
    concatenated_building['temperature'].iloc[-4:] = predicted_temps
    
    return concatenated_building


In [33]:
resulting_dataframes_100_revised = [process_building_data(data, test_data, i) for i in range(1, 101)]

/var/folders/__/r8r58jz17gj32tjwycl0m0hr0000gn/T/ipykernel_80346/4196958054.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_building[column].iloc[-4:] = concatenated_building[column].iloc[-28:-24].values
/var/folders/__/r8r58jz17gj32tjwycl0m0hr0000gn/T/ipykernel_80346/4196958054.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_building[column].iloc[-4:] = concatenated_building[column].iloc[-28:-24].values
/var/folders/__/r8r58jz17gj32tjwycl0m0hr0000gn/T/ipykernel_80346/4196958054.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the

In [34]:
final_data_100_revised = pd.concat(resulting_dataframes_100_revised, axis=0, ignore_index=True)

In [40]:
len(final_data_100_revised)

218588

In [35]:
last_7_days_data = final_data_100_revised[final_data_100_revised['date_time'] > '2022-08-24 23:00:00'].copy()
prior_data = final_data_100_revised[final_data_100_revised['date_time'] <= '2022-08-24 23:00:00'].copy()

In [36]:
last_7_days_data

,date_time,building_number,temperature,rainfall,windspeed,humidity,power_consumption,building_type,total_area,cooling_area,...,week,sin_time,cos_time,holiday,peak,no_peak,upper_outliers,lower_outliers,outliers,anomaly
2036,2022-08-25 00:00:00,1.0,23.500000,0.0,2.2,72.0,2208.96,Other Buildings,110634.00,39570.00,...,34.0,0.000000,1.000000,0.0,0.0,1.0,NaN,NaN,NaN,NaN
2037,2022-08-25 01:00:00,1.0,23.000000,0.0,0.9,72.0,1806.24,Other Buildings,110634.00,39570.00,...,34.0,0.258819,0.965926,0.0,0.0,1.0,NaN,NaN,NaN,NaN
2038,2022-08-25 02:00:00,1.0,22.700000,0.0,1.5,75.0,1387.20,Other Buildings,110634.00,39570.00,...,34.0,0.500000,0.866025,0.0,0.0,1.0,NaN,NaN,NaN,NaN
2039,2022-08-25 03:00:00,1.0,22.100000,0.0,1.3,78.0,1925.28,Other Buildings,110634.00,39570.00,...,34.0,0.707107,0.707107,0.0,0.0,1.0,NaN,NaN,NaN,NaN
2040,2022-08-25 04:00:00,1.0,21.800000,0.0,1.0,77.0,NaN,Other Buildings,110634.00,39570.00,...,34.0,0.866025,0.500000,0.0,0.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218583,2022-08-31 23:00:00,100.0,19.100000,0.0,0.1,99.0,NaN,Hotel and Resort,57497.84,40035.23,...,35.0,-0.258819,0.965926,0.0,0.0,0.0,NaN,NaN,NaN,NaN
218584,2022-08-31 23:00:00,100.0,22.204283,0.0,0.1,99.0,NaN,Hotel and Resort,57497.84,40035.23,...,35.0,0.000000,1.000000,0.0,0.0,1.0,NaN,NaN,NaN,NaN
218585,2022-08-31 23:00:00,100.0,21.864901,0.0,0.1,99.0,NaN,Hotel and Resort,57497.84,40035.23,...,35.0,0.258819,0.965926,0.0,0.0,1.0,NaN,NaN,NaN,NaN
218586,2022-08-31 23:00:00,100.0,21.637933,0.0,0.1,99.0,NaN,Hotel and Resort,57497.84,40035.23,...,35.0,0.500000,0.866025,0.0,0.0,1.0,NaN,NaN,NaN,NaN


In [23]:
last_7_days_data.drop(['date_time', 'building_type', 'total_area', 'cooling_area', 'solar_power_capacity','upper_outliers', 'lower_outliers', 'outliers',
       'anomaly'], axis =1 , inplace=True)

In [24]:
prior_data.drop(['date_time', 'building_type', 'total_area', 'cooling_area', 'solar_power_capacity','upper_outliers', 'lower_outliers', 'outliers',
       'anomaly'], axis =1 , inplace=True)

In [25]:
last_7_days_data['day'] = last_7_days_data['day'].astype(int)
last_7_days_data['week_of_month'] = last_7_days_data['week_of_month'].astype(int)
last_7_days_data['rainfall_any'] = last_7_days_data['rainfall_any'].astype(int)
last_7_days_data['holiday'] = last_7_days_data['holiday'].astype(int)
last_7_days_data['peak'] = last_7_days_data['peak'].astype(int)
last_7_days_data['no_peak'] = last_7_days_data['no_peak'].astype(int)

In [26]:
prior_data['day'] = prior_data['day'].astype(int)
prior_data['week_of_month'] = prior_data['week_of_month'].astype(int)
prior_data['rainfall_any'] = prior_data['rainfall_any'].astype(int)
prior_data['holiday'] = prior_data['holiday'].astype(int)
prior_data['peak'] = prior_data['peak'].astype(int)
prior_data['no_peak'] = prior_data['no_peak'].astype(int)

In [37]:
len(last_7_days_data)

17200

In [58]:
last_7_days_data.to_csv('last_test.csv', index=False)

In [59]:
prior_data.to_csv('last_train.csv', index=False)

In [60]:
last_7_days_data.columns.nunique()

20

In [61]:
prior_data.columns.nunique()

20